In [ ]:
!pip install einops
!pip install kornia
import torch.nn as nn
import torch.nn.functional as F
from contextlib import contextmanager
import torchvision
import torch
from torchvision.datasets import MNIST,CIFAR10
import matplotlib.pyplot as plt
from six import add_metaclass
from torch.nn import init
import torch.optim as optim
import os
from torch.utils.data import DataLoader,random_split
from torchvision.utils import save_image
import numpy as np
from PIL import Image
from torchvision import datasets, transforms
from contextlib import contextmanager
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
from six import add_metaclass
from torch.optim import Adam, lr_scheduler

from kornia.augmentation import RandomCrop, Normalize
from argparse import ArgumentParser
import errno
from einops import rearrange

In [ ]:
def weights_init(m):
    # Code taken from https://discuss.pytorch.org/t/initializing-rnn-gru-and-lstm-correctly/23605/8
    parameters = m.state_dict()
    for each_key in parameters.keys():
        print(f'Init-{each_key}')
        if 'weight_ih' in each_key:
            nn.init.orthogonal_(parameters[each_key])
        elif 'weight_hh' in each_key:
            nn.init.orthogonal_(parameters[each_key])
        elif 'bias' in each_key:
            nn.init.constant_(parameters[each_key], val=0)

In [ ]:
class ReNet(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size=(2, 2), rnn='GRU', depth=(1,1)):
        super(ReNet, self).__init__()
        if rnn == 'GRU':
            rnn = nn.GRU
        elif rnn == 'LSTM':
            rnn = nn.LSTM
        
        self.lstm_h = rnn(input_size, hidden_size, bias=False, num_layers=depth[0], bidirectional=True)
        self.lstm_v = rnn(hidden_size * 2, hidden_size, bias=False, num_layers=depth[1], bidirectional=True)

        if isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            self.kernel_size = kernel_size
        
        self.lstm_h.apply(weights_init)
        self.lstm_v.apply(weights_init)

    def forward(self, x):
        k_w, k_h = self.kernel_size
        b, c, h, w = x.size()
        assert h % k_h == 0 and w % k_w == 0, 'input size does not match with kernel size'
        x = rearrange(x, 'b c (h1 h2) (w1 w2) -> h1 (b w1) (c h2 w2)', w2=k_w, h2=k_h)
        x, _ = self.lstm_h(x)
        x = rearrange(x, 'h1 (b w1) (c h2 w2) -> w1 (b h1) (c h2 w2)', b=b, w2=k_w, h2=k_h)
        x, _ = self.lstm_v(x)
        x = rearrange(x, 'w1 (b h1) (c h2 w2) -> b (c h2 w2) h1 w1', b=b, w2=k_w, h2=k_h)
        return x


In [ ]:
renet = nn.Sequential(
    ReNet(2 * 2 * 3, 128, kernel_size=(2, 2)), 
    ReNet(2 * 2 * 256, 128, kernel_size=(2, 2)),
    nn.Flatten(),
    nn.Linear(256 * 8 * 8, 4096),
    nn.ReLU(),
    nn.Linear(4096, 10),
)
device = torch.device('cuda:0')
renet = renet.to(device)

In [ ]:
transform_list = [
      transforms.Pad(padding=4, padding_mode='reflect'),
      transforms.RandomCrop(32),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
]

CIFAR_dataset_train = CIFAR10('./data',train=True,download=True,transform=transforms.Compose(transform_list))
CIFAR_dataset_test = CIFAR10('./data',train=False,download=True,transform=transforms.Compose(transform_list))

In [ ]:
cifar_train_loader = DataLoader(CIFAR_dataset_train,shuffle=True,batch_size=128,pin_memory=True)
cifar_test_loader = DataLoader(CIFAR_dataset_test,shuffle=True,batch_size=128,pin_memory=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_epochs = 20
learning_rate = 0.01
optimizer = torch.optim.SGD(renet.parameters(), lr=learning_rate)  
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
    for images,labels in cifar_train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = renet(images)

        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    correct = 0
    total = 0
    accuracy = 0
    for images, labels in cifar_test_loader:
        images, labels = images.to(device), labels.to(device)
            
        outputs = renet(images)
            
        predictions = torch.argmax(outputs, dim=1)
        correct += (predictions == labels).sum()
            
        total += len(labels)
            
        accuracy = correct * 100 / total
    print("Epoch: {}, Test Accuracy: {}%".format(epoch, accuracy))